## Importing Dependencies

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RNN
from keras.utils import np_utils

Using TensorFlow backend.


## Loading of Data

In [9]:
text = (open("sonnets.txt").read())
text=text.lower()
print(sorted(list(set(text))))

['\n', ' ', '!', "'", '(', ')', ',', '-', '.', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


## Creating character/word mappings

In [10]:
characters = sorted(list(set(text)))

n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

## Data pre-processing

In [13]:
X = []
Y = []
length = len(text)
seq_length = 100

for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

print(X[0])

[38, 20, 0, 0, 1, 17, 29, 26, 24, 1, 17, 12, 20, 29, 16, 30, 31, 1, 14, 29, 16, 12, 31, 32, 29, 16, 30, 1, 34, 16, 1, 15, 16, 30, 20, 29, 16, 1, 20, 25, 14, 29, 16, 12, 30, 16, 6, 0, 1, 31, 19, 12, 31, 1, 31, 19, 16, 29, 16, 13, 36, 1, 13, 16, 12, 32, 31, 36, 3, 30, 1, 29, 26, 30, 16, 1, 24, 20, 18, 19, 31, 1, 25, 16, 33, 16, 29, 1, 15, 20, 16, 6, 0, 1, 13, 32, 31, 1, 12, 30]


In [17]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)
print(Y_modified[0])


[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## A baseline model

In [135]:
model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [138]:
model.fit(X_modified, Y_modified, epochs=1, batch_size=100)

model.save_weights('/Users/pranjal/Desktop/text_generator/models/text_generator_400_0.2_400_0.2_baseline.h5')

Epoch 1/1
97820/97820 [==============================] - 12269s 125ms/step - loss: 2.8514


In [139]:
model.load_weights('/Users/pranjal/Desktop/text_generator/models/text_generator_400_0.2_400_0.2_baseline.h5')


## Generating Text

In [140]:
string_mapped = X[99]
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

In [141]:
#combining text
txt=""
for char in full_string:
    txt = txt+char
txt

's the riper should by time decease,\n his tender heir might bear his memory:\n but thou, contracted toet she the the the the the the the the \n      thi the the the the the the the the the the the the the the the the the \n      thi the the the the the the the the the the the the the the the the the \n      thi the the the the the the the the the the the the the the the the the \n      thi the the the the the the the the the the the the the the the the the \n      thi the the the the the the the the th'